<a href="https://colab.research.google.com/github/snvssk/data298A/blob/development/SatelliteImagery/DataProcessingS2LookingBoundingBoxCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from skimage.measure import label, regionprops, find_contours



In [ ]:
images = sorted(glob(os.path.join("./train/", "train_image", "*")))
masks = sorted(glob(os.path.join("./train/", "train_mask", "*")))

In [ ]:
images[1]

'/content/drive/MyDrive/SmallDataset/train/train_image/train_10.png'

In [ ]:
masks[1]

'/content/drive/MyDrive/SmallDataset/train/train_mask/train_10.png'

In [ ]:
""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)



In [ ]:
""" Convert a mask to border image """
def mask_to_border(mask):
    h, w = mask.shape
    border = np.zeros((h, w))

    contours = find_contours(mask, 128)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255

    return border



In [ ]:
""" Mask to bounding boxes """
def mask_to_bbox(mask):
    bboxes = []

    mask = mask_to_border(mask)
    lbl = label(mask)
    props = regionprops(lbl)
    for prop in props:
        x1 = prop.bbox[1]
        y1 = prop.bbox[0]

        x2 = prop.bbox[3]
        y2 = prop.bbox[2]

        bboxes.append([x1, y1, x2, y2])

    return bboxes



In [ ]:
def parse_mask(mask):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    return mask



In [ ]:
create_dir("results")

#  Loop over the dataset
for x, y in tqdm(zip(images, masks), total=len(images)):

  # Extract the name
  name = x.split("/")[-1].split(".")[0]

  #  Read image and mask 
  x = cv2.imread(x, cv2.IMREAD_COLOR)
  y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

  #  Detecting bounding boxes 
  bboxes = mask_to_bbox(y)

  #  marking bounding box on image 
  for bbox in bboxes:
      x = cv2.rectangle(x, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)

        # """ Saving the image """
      cat_image = np.concatenate([x, parse_mask(y)], axis=1)
      cv2.imwrite(f"results/{name}.png", cat_image)

100%|██████████| 25/25 [01:12<00:00,  2.90s/it]


In [ ]:
!pip install image_bbox_slicer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for image-bbox-slicer: filename=image_bbox_slicer-0.4-py3-none-any.whl size=12151 sha256=a74298435d1e91209e2933c81162f0b08b51197bbdc8e64660d076fb549daf50
  Stored in directory: /root/.cache/pip/wheels/9a/57/4f/559648fffb85e4da4d4f1ff1660372e5f4df5f5db96b8d7086
Successfully built image-bbox-slicer
